In [1]:
#Coursework 1

#To-Do (fun messing about afterwards):
#- Limit amount of deliveries.
#- Clean up the code. 
#   - Tbh half of this isn't needed, perceptdirection does everything percept closest does anyway.
#- Instead of just going towards the closest within sight, bot goes towards closest one it 
#knows about, and just assigns any in sight to the list for later.
#- Fix dynamic table thing
#- Successfully implement OWR.
#- Port to VS Code, create a graphical thing where the table updates in front of you, instead
#of creating a new one each time.

#Bugs Found
#Occasionally some coordinates come up as "unknown", need to make code to check for them.

In [2]:
import random
import numpy as np

In [3]:
class EnvironmentClass:
    def __init__(self):
        #sets up the environment.
        choice = 0
        a = 0
        print("How big would you like the environment to be? (2-8)")
        self.size = self.validateint(2, 8)
        self.size = int(self.size)
        while a < 1:
            choice = input("Do you want deliveries to be randomized or picked?\n 1. Randomized.\n2. Picked.\n")
            choice = int(choice)
            if choice < 0 or choice > 3:
                print("Enter a number between 1 or 2") 
            if choice == 1:
                self.EnvGen = [[random.choice(["1", "2"]) for _ in range(self.size)] for _ in range(self.size)]
                break
            if choice == 2:
                self.EnvGen = [["0" for _ in range(self.size)] for _ in range(self.size)]
                amount = 0
                print("How many deliveries would you like? (5-20")
                amount = self.validateint(5, 20)
                amount = int(amount)
                print("Where would you like them to go?")
                b = 0
                while b < amount:
                    b+=1
                    print(f"{b}:\nPick the x coordinate. Available: 0 - {self.size - 1}")
                    x = self.validateint(0, self.size - 1)
                    print(f"Now the y coordinate. Available: 0 - {self.size - 1}")
                    y = self.validateint(0, self.size - 1)
                    self.EnvGen[y][x] = "2"
                    break
        i = 0
        num = 0
        while i < self.size:
            j = 0
            while j < self.size:
                if self.EnvGen[i][j] == "2":
                    self.EnvGen[i][j] = "2"
                else:
                    num = random.choice(["1", "2", "2", "3" "3", "4", "4", "5", "5", "6"])
                    if num == "1":
                        self.EnvGen[i][j] = "3"
                j+=1
            i+=1
        #self.addOWR()
        #Clear = 1
        #Delivery = 2
        #Blockade = 3
        #One Way Road = 4, 5 (5 being where the robot can enter from)
        #Topright = [0,0]

    def addOWR(self):
        #creating a function specifically for this. was going to make road length random, but was too complicated and unneccessary.
        pointx = 0
        pointy = 0
        i = 0
        while i < 1:
            direction = random.choice(["Left", "Right", "Up", "Down"])
            if direction == "Left":
                pointx = random.randrange(2, self.size - 1)
                pointy = random.randrange(0, self.size - 1)
                if self.EnvGen[pointy][pointx] == "2" or self.EnvGen[pointy][pointx - 1] == "2" or self.EnvGen[pointy][pointx + 1] == "3":
                    continue
                else: 
                    self.EnvGen[pointy][pointx] = "5"
                    self.EnvGen[pointy][pointx - 1] = "4"
                    break
            elif direction == "Right":
                pointx = random.randrange(0, 2)
                pointy = random.randrange(0, self.size - 1)
                if self.EnvGen[pointy][pointx] == "2" or self.EnvGen[pointy][pointx + 1] == "2" or self.EnvGen[pointy][pointx - 1] == "3":
                    continue
                else:
                    self.EnvGen[pointy][pointx] = "5"
                    self.EnvGen[pointy][pointx + 1] = "4"
                    break
            elif direction == "Up":
                pointx = random.randrange(0, self.size - 1)
                pointy = random.randrange(2, self.size - 1)
                if self.EnvGen[pointy][pointx] == "2" or self.EnvGen[pointy - 1][pointx] == "2" or self.EnvGen[pointy + 1][pointx] == "3":
                    continue
                else:
                    self.EnvGen[pointy][pointx] = "5"
                    self.EnvGen[pointy - 1][pointx] = "4"
                    break
            elif direction == "Down":
                pointx = random.randrange(0, self.size - 1)
                pointy = random.randrange(0, 2)
                if self.EnvGen[pointy][pointx] == "2" or self.EnvGen[pointy + 1][pointx] == "2" or self.EnvGen[pointy - 1][pointx] == "3":
                    continue
                else:
                    self.EnvGen[pointy][pointx] = "5"
                    self.EnvGen[pointy + 1][pointx] = "4"
                    break
    
    def validateint(self, minint, maxint):
        intinput = input()
        intinput = int(intinput)
        while intinput > maxint or intinput < minint:
            if intinput > maxint:
                print(f"Please input a number lower than {maxint + 1}.")
            if intinput < minint:
                print(f"Please input a number higher than {minint - 1}.")
            intinput = input()
            intinput = int(intinput)
        return intinput

In [4]:
class Robot:
    def __init__(self, environment):
        #initialization for the ai.
        self.env = environment
        self.Table = [["0" for _ in range(self.env.size)] for _ in range(self.env.size)]
        self.dynamicTable = [["Unknown " for _ in range(self.env.size)] for _ in range(self.env.size)]
        self.size = self.env.size - 1
        self.deliveries = 0
        self.totaldeliveries = 0
        self.totalmoves = 0
        self.lastmovex = 0
        self.lastmovey = 0
        self.moved = "No"
        #memory is deliveries it has seen. total memory is the same, but just for show + none are deleted.
        self.memory = []
        self.totalmemory = []
        int(self.size)
        i = 0
        while i == 0:
            if i == 0:
                print("Would you like to choose or randomize the starting position? \n1: Choose\n2: Randomize.")
                choice = self.env.validateint(1, 2)
            if choice == 1:
                self.xlocation, self.ylocation = self.getStartingPositionChoice()
            elif choice == 2:
                self.xlocation, self.ylocation = self.getStartingPositionRandom()
            if self.env.EnvGen[self.ylocation][self.xlocation] == "3":
                print("Can't start here. Building.")
            else:
                break
        print(f"\nRobot started in [{self.xlocation},{self.ylocation}].")

    def getStartingPositionChoice(self):
        print(F"Please input the x coordinate. \nAvailable: 0 - {self.size}.")
        x = self.env.validateint(0, self.size)
        print(f"Now the y coordinate. \nAvailable: 0 - {self.size}.")
        y = self.env.validateint(0, self.size)
        return x, y

    def getStartingPositionRandom(self):
        x = random.randrange(0, self.size)
        y = random.randrange(0, self.size)
        return x, y
    
    def percept(self):
        #Checks the space its on
        return self.xlocation, self.ylocation, self.env.EnvGen[self.ylocation][self.xlocation]

    def perceptclosest(self):
        #checks if there are any deliveries close to it
        #Initialization
        self.locationleft = self.xlocation
        self.locationright = self.xlocation
        self.locationup = self.ylocation
        self.locationdown = self.ylocation
        #Left
        if self.xlocation == 0:
            self.locationleft = self.xlocation
        else: 
            self.locationleft = self.xlocation - 1
        #Right
        if self.xlocation == self.size:
            self.locationright = self.xlocation
        else:
            self.locationright = self.xlocation + 1
        #Up
        if self.ylocation == 0:
            self.locationup = self.ylocation
        else:
            self.locationup = self.ylocation - 1
        #Down
        if self.ylocation == self.size:
            self.locationdown = self.ylocation
        else:
            self.locationdown = self.ylocation + 1
        return self.env.EnvGen[self.ylocation][self.locationleft], self.env.EnvGen[self.ylocation][self.locationright], self.env.EnvGen[self.locationup][self.xlocation], self.env.EnvGen[self.locationdown][self.xlocation]

    def perceptclosestcorners(self):
        #checks if there are any deliveries close, in the corners
        #Initialization
        self.locationleftul = self.xlocation
        self.locationupul = self.ylocation
        self.locationrightur = self.xlocation
        self.locationupur = self.ylocation
        self.locationleftdl = self.xlocation
        self.locationdowndl = self.ylocation
        self.locationrightdr = self.xlocation
        self.locationdowndr = self.ylocation
        #Up-Left
        if self.xlocation == 0:
            self.locationleftul = self.xlocation
        elif self.ylocation == 0:
            self.locationupul = self.ylocation
        else:
            self.locationleftul = self.xlocation - 1
            self.locationupul = self.ylocation - 1 
        #Up-Right
        if self.xlocation == self.size:
            self.locationrightur = self.xlocation
        elif self.ylocation == 0:
            self.locationupur = self.ylocation
        else:
            self.locationrightur = self.xlocation + 1
            self.locationupur = self.ylocation - 1
        #Down-Left
        if self.xlocation == 0:
            self.locationleftdl = self.xlocation
        elif self.ylocation == self.size:
            self.locationdowndl = self.ylocation
        else:
            self.locationleftdl = self.xlocation - 1
            self.locationdowndl = self.ylocation + 1
        #Down-Right
        if self.xlocation == self.size:
            self.locationrightdr = self.xlocation 
        elif self.ylocation == self.size:
            self.locationdowndr = self.ylocation
        else:
            self.locationrightdr = self.xlocation + 1
            self.locationdowndr = self.ylocation + 1
        return self.env.EnvGen[self.locationupul][self.locationleftul], self.env.EnvGen[self.locationupur][self.locationrightur], self.env.EnvGen[self.locationdowndl][self.locationleftdl], self.env.EnvGen[self.locationdowndr][self.locationrightdr]
    
    def perceptdirection(self):
       #once percept closest fails, searches further for the closest delivery and makes a decision based on the data.
        left = 0
        right = 0
        up = 0
        down = 0
        steps = 0
        direction = 0
        locationleft = self.xlocation - 1
        locationright = self.xlocation + 1
        locationup = self.ylocation - 1
        locationdown = self.ylocation + 1
        #Check left
        if self.xlocation > 0 and locationleft > 0:
            while locationleft > 0:
                steps += 1
                locationleft = self.xlocation - steps
                if 0 <= locationleft <= self.size and self.env.EnvGen[self.ylocation][locationleft] == "2":
                    print(steps)
                    self.MemoryAdd(locationleft, self.ylocation)
                    left = steps
                    break
                elif locationleft <= 0 and (self.env.EnvGen[self.ylocation][locationleft] == "1" or self.env.EnvGen[self.ylocation][locationleft] == "3"):
                    print("hit left walL")
                    left = self.size + 1
                    break
        else: 
            left = self.size + 1
        #Check Right
        steps = 0
        if self.xlocation < self.size and locationright < self.size:
            while locationright < self.size:
                steps += 1
                locationright = self.xlocation + steps
                if 0 <= locationright <= self.size and self.env.EnvGen[self.ylocation][locationright] == "2":
                    print(steps)
                    self.MemoryAdd(locationright, self.ylocation)
                    right = steps
                    break
                elif locationright >= self.size and (self.env.EnvGen[self.ylocation][locationright] == "1" or self.env.EnvGen[self.ylocation][locationright] == "3"):
                    print("hit right wall")
                    right = self.size + 1
                    break
        else:
            right = self.size + 1
        #Check Up
        steps = 0
        if self.ylocation > 0 and locationup > 0:
            while locationup > 0:
                steps += 1
                locationup = self.ylocation - steps
                if 0 <= locationup <= self.size and self.env.EnvGen[locationup][self.xlocation] == "2":
                    up = steps
                    print(up)
                    self.MemoryAdd(self.xlocation, locationup)
                    break
                elif locationup <= 0 and (self.env.EnvGen[locationup][self.xlocation] == "1" or self.env.EnvGen[locationup][self.xlocation] == "3"):
                    print("hit ceiling")
                    up = self.size + 1
                    break
        else:
            up = self.size+1
        #Check Down
        steps = 0
        if self.ylocation < self.size and locationdown < self.size:
            while locationdown < self.size:
                steps += 1
                locationdown = self.ylocation + steps
                if 0 <= locationdown <= self.size and self.env.EnvGen[locationdown][self.xlocation] == "2":
                    print(steps)
                    down = steps
                    self.MemoryAdd(self.xlocation, locationdown)
                    break
                elif locationdown >= self.size and (self.env.EnvGen[locationdown][self.xlocation] == "1" or self.env.EnvGen[locationdown][self.xlocation] == "3"):
                    print("hit lowest wall")
                    down = self.size + 1
                    break
        else:
            down = self.size + 1
        #Determine closest delivery
        if left < right and left < up and left < down:
            #go left
            direction = "Left"
            print("left")
            return direction, left
        elif right < left and right < up and right < down:
            #go right
            direction = "Right"
            print("right")
            return direction, right
        elif up < down and up < left and up < right:
            #go up
            direction = "Up"
            print("up")
            return direction, up
        elif down < up and down < left and down < right:
            #go down
            direction = "Down"
            print("down")
            return direction, down
        else:
            #add the corner ones here
            print("corner time")
            direction = "None"
            steps = 1
            print(direction)
            return direction, steps

    def perceptdirectioncorners(self):
        #if percept direction fails, this searches the corner directions.
        upleft = 0
        upright = 0
        downleft = 0
        downright = 0
        steps = 0
        direction = 0
        locationleft = self.xlocation - 1
        locationright = self.xlocation + 1
        locationup = self.ylocation - 1
        locationdown = self.ylocation + 1
        #check upleft
        if self.xlocation > 0 and locationleft > 0 and self.ylocation > 0 and locationup > 0:
            while locationleft > 0 and locationup > 0:
                steps+= 1
                locationleft = self.xlocation - steps
                locationup = self.ylocation - steps
                if 0 <= locationleft <= self.size and 0 <= locationup <= self.size and self.env.EnvGen[locationup][locationleft] == "2":
                    print(steps)
                    self.MemoryAdd(locationleft, locationup)
                    upleft = steps
                    break
                elif locationleft <= 0 and locationup <= 0 and (self.env.EnvGen[locationup][locationleft] == "1" or self.env.EnvGen[locationup][locationleft] == "3"):
                    print("hit upleft wall")
                    upleft = self.size + 1
                    break
        else:
            upleft = self.size+1
        #Check upright
        steps = 0
        locationleft = self.xlocation - 1
        locationup = self.xlocation - 1
        if self.xlocation < self.size and locationright < self.size and self.ylocation > 0 and locationup > 0:
            while locationright < self.size and locationup > 0:
                steps+=1
                locationright = self.xlocation + steps
                locationup = self.ylocation - steps
                if 0 <= locationright <= self.size and 0 <= locationup <= self.size and self.env.EnvGen[locationup][locationright] == "2":
                    print(steps)
                    upright = steps
                    self.MemoryAdd(locationright, locationup)
                    break
                elif locationright >= self.size and locationup <= 0 and (self.env.EnvGen[locationup][locationright] == "1" or self.env.EnvGen[locationup][locationright] == "3"):
                    print("hit upright wall")
                    upright = self.size + 1
                    break
        else:
            upright = self.size + 1
        locationright = self.xlocation + 1
        locationup = self.ylocation - 1
        #check downleft
        steps = 0
        if self.xlocation > 0 and locationleft > 0 and self.ylocation < self.size and locationdown < self.size:
            while locationleft > 0 and locationdown < self.size:
                steps+= 1
                locationleft = self.xlocation - steps
                locationdown = self.ylocation + steps
                if 0 <= locationleft <= self.size and 0 <= locationdown <= self.size and self.env.EnvGen[locationdown][locationleft] == "2":
                    print(steps)
                    downleft = steps
                    self.MemoryAdd(locationleft, locationdown)
                    break
                elif locationleft <= 0 and locationdown >= self.size and (self.env.EnvGen[locationdown][locationleft] == "1" or self.env.EnvGen[locationdown][locationleft] == "3"):
                    print("hit downleft wall")
                    downleft = self.size + 1
                    break
        else:
            downleft = self.size + 1
        locationleft = self.xlocation - 1
        locationdown = self.ylocation + 1
        #check downright
        steps = 0
        if self.xlocation < self.size and locationright < self.size and self.ylocation < self.size and locationdown < self.size:
            while locationright < self.size and locationdown < self.size:
                steps +=1
                locationright = self.xlocation + steps
                locationdown = self.ylocation + steps
                if 0 <= locationright <= self.size and 0 <= locationdown <= self.size and self.env.EnvGen[locationdown][locationright] == "2":
                    print(steps)
                    downright = steps
                    self.MemoryAdd(locationright, locationdown)
                    break
                elif locationright >= self.size and locationdown >= self.size and (self.env.EnvGen[locationdown][locationright] == "1" or self.env.EnvGen[locationdown][locationright] == "3"):
                    print("hit downright wall")
                    downleft = self.size + 1
                    break
        else:
            downright = self.size + 1
        #determine closest delivery
        if upleft < upright and upleft < downleft and upleft < downright:
            #go upleft
            direction = "Upleft"
            print("upleft")
            return direction, upleft
        elif upright < upleft and upright < downleft and upright < downright:
            #go upright
            direction = "Upright"
            print("Upright")
            return direction, upright
        elif downleft < upleft and downleft < upright and downleft < downright:
            #go downleft
            direction = "Downleft"
            print("downleft")
            return direction, downleft
        elif downright < upleft and downright < upright and downright < downleft:
            #go downright
            direction = "Downright"
            print("downright")
            return direction, downright
        else:
            print("random")
            direction = "Random"
            steps = 1
            print("random")
            return direction, steps

    def MemorySearch(self, x, y): 
        j = 0
        dimension = self.size + 1
        if 0 <= x <= self.size and 0 <= y <= self.size:
            j = y * dimension + x + 1
            print(f"Testing - {j}")
            return j
        else:
            return -1 

    def MemoryAdd(self, x, y):
        delivery = self.MemorySearch(x, y)
        if delivery in self.memory:
            return -1
        else:
            self.memory.append(delivery)
            self.totalmemory.append(delivery)

    def MemoryRemove(self, x, y):
        delivery = self.MemorySearch(x, y)
        if delivery in self.memory:
            self.memory.remove(delivery)
        else: 
            return -1

    def deliver(self, x, y):
        #delivery, just changes deliver to clear
        print(f"Delivering at [{x},{y}].")
        self.env.EnvGen[self.ylocation][self.xlocation] = "1"
        self.deliveries += 1
        self.MemoryRemove(x, y)

    def move(self, direction):
        #Controls x axis movement, left and right, or y axis, up and down
        self.lastmovex = self.xlocation
        self.lastmovey = self.ylocation
        self.moved = "Yes"
        self.totalmoves += 1
        if direction == "Left":
            if self.xlocation == 0:
                return self.xlocation
            else:
                self.xlocation = self.xlocation - 1
                self.lastmove = "Left"
                return self.xlocation
        elif direction == "Right":
            if self.xlocation == self.size:
                return self.xlocation
            else:
                self.xlocation = self.xlocation + 1
                self.lastmove = "Right"
                return self.xlocation
        elif direction == "Up":
            if self.ylocation == 0:
                return self.ylocation
            else:
                self.ylocation = self.ylocation - 1
                self.lastmove = "Up"
                return self.ylocation
        elif direction == "Down":
            if self.ylocation == self.size:
                return self.ylocation
            else:
                self.ylocation = self.ylocation + 1
                self.lastmove = "Down"
                return self.ylocation

    def movexy(self, direction):
        self.lastmovex = self.xlocation
        self.lastmovey = self.ylocation
        self.moved = "Yes"
        self.totalmoves += 1
        #controls corner movement, upright, downleft and the other 2
        if direction == "Upleft":
            if self.ylocation == 0 or self.xlocation == 0:
                return self.xlocation, self.ylocation
            else:
                self.xlocation = self.xlocation - 1
                self.ylocation = self.ylocation - 1
                return self.xlocation, self.ylocation
        elif direction == "Upright":
            if self.ylocation == 0 or self.xlocation == self.size:
                return self.xlocation, self.ylocation
            else:
                self.xlocation = self.xlocation + 1
                self.ylocation = self.ylocation - 1
                return self.xlocation, self.ylocation
        elif direction == "Downleft":
            if self.ylocation == self.size or self.xlocation == 0:
                return self.xlocation, self.ylocation
            else:
                self.xlocation = self.xlocation - 1
                self.ylocation = self.ylocation + 1
                return self.xlocation, self.ylocation
        elif direction == "Downright":
            if self.ylocation == self.size or self.xlocation == self.size:
                return self.xlocation, self.ylocation
            else:
                self.xlocation = self.xlocation + 1
                self.ylocation = self.ylocation + 1
                return self.xlocation, self.ylocation
    
    def act(self, perception):
        x, y, status = perception
        direction = 0
        steps = 0
        i = 0
        if status == "2":
            self.MemoryAdd(x, y)
            self.deliver(x, y)
        elif status == "1":
            #code searching the four coordinates next to it (percept closest)
            left, right, up, down = self.perceptclosest()
            if self.env.EnvGen[self.xlocation][self.ylocation] == "4":
                #to make sure it actually follows the one way road.
                if left == "5":
                    self.xlocation = self.move("Left")
                elif right == "5":
                    self.xlocation = self.move("Right")
                elif up == "5":
                    self.ylocation = self.move("Up")
                elif down == "5":
                    self.ylocation = self.move("Down")
            elif left == "2":
                self.xlocation = self.move("Left")
                self.MemoryAdd(self.xlocation, self.ylocation)
            elif right == "2":
                self.xlocation = self.move("Right")
                self.MemoryAdd(self.xlocation, self.ylocation)
            elif up == "2":
                self.ylocation = self.move("Up")
                self.MemoryAdd(self.xlocation, self.ylocation)
            elif down == "2":
                self.ylocation = self.move("Down")
                self.MemoryAdd(self.xlocation, self.ylocation)
            else:
                upleft, upright, downleft, downright = self.perceptclosestcorners()
                #code moving to corners (perceptclosestcorners)
                if upleft == "2":
                    self.xlocation, self.ylocation = self.movexy("Upleft")
                    self.MemoryAdd(self.xlocation, self.ylocation)
                elif upright == "2":
                    self.xlocation, self.ylocation = self.movexy("Upright")
                    self.MemoryAdd(self.xlocation, self.ylocation)
                elif downleft == "2":
                    self.xlocation, self.ylocation = self.movexy("Downleft")
                    self.MemoryAdd(self.xlocation, self.ylocation)
                elif downright == "2":
                    self.xlocation, self.ylocation = self.movexy("Downright")
                    self.MemoryAdd(self.xlocation, self.ylocation)
                else:
                    self.act2()

    def act2(self):
    #code searching (perceptdirection)
        direction, steps = self.perceptdirection()
        left, right, up, down = self.perceptclosest()
        if direction == "Left" or (self.env.EnvGen[self.ylocation][self.xlocation] == "5" and self.env.EnvGen[self.ylocation][self.xlocation - 1] == "4"):
            if left == "3":
                direction = "None"
                print("left blocked")
            else:
                print("left")
                self.xlocation = self.move("Left")
        elif direction == "Right" or (self.env.EnvGen[self.ylocation][self.xlocation] == "5" and self.env.EnvGen[self.ylocation][self.xlocation + 1] == "4"):
            if right == "3":
                direction = "None"
                print("right blocked")
            else:
                print("right")
                self.xlocation = self.move("Right")
        elif direction == "Up" or (self.env.EnvGen[self.ylocation][self.xlocation] == "5" and self.env.EnvGen[self.ylocation - 1][self.xlocation] == "4"):
            if up == "3":
                direction = "None"
                print("up blocked")
            else:
                print("up")
                self.ylocation = self.move("Up")
        elif direction == "Down" or (self.env.EnvGen[self.ylocation][self.xlocation] == "5" and self.env.EnvGen[self.ylocation + 1][self.xlocation] == "4"):
            if down == "3":
                direction = "None"
                print("down blocked")
            else:
                print("down")
                self.ylocation = self.move("Down")
        if direction == "None":
            direction, steps = self.perceptdirectioncorners()
            upleft, upright, downleft, downright = self.perceptclosestcorners()
            if direction == "Upleft":
                if upleft == "3":
                    direction = "Random"
                    print("upleft blocked")
                else:
                    print("upleft")
                    self.xlocation, self.ylocation = self.movexy("Upleft")
            elif direction == "Upright":
                if upright == "3":
                    direction = "Random"
                    print("upright blocked")
                else:
                    print("upright")
                    self.xlocation, self.ylocation = self.movexy("Upright")
            elif direction == "Downleft":
                if downleft == "3":
                    direction = "Random"
                    print("downleft blocked")
                else:
                    print("downleft")
                    self.xlocation, self.ylocation = self.movexy("Downleft")
            elif direction == "Downright":
                if downright == "3":
                    direction == "Random"
                    print("downright blocked")
                print("downright")
                self.xlocation, self.ylocation = self.movexy("Downright")
            if direction == "Random":
                print("random")
                self.actRandom()
        
    def actRandom(self):
        valid = "No"
        while valid == "No":
            direction = random.choice(["Left", "Right", "Up", "Down"])
            left, right, up, down = self.perceptclosest()
            if direction == "Left":
                valid = self.randomCheck(left)
            elif direction == "Right":
                valid = self.randomCheck(right)
            elif direction == "Up":
                valid = self.randomCheck(up)
            elif direction == "Down":
                valid = self.randomCheck(down)
            if valid == "Yes":
                break
        if direction == "Left" or direction == "Right":
            self.xlocation == self.move(direction)
        elif direction == "Up" or direction == "Down":
            self.ylocation == self.move(direction)

    def randomCheck(self, randirection):
        if randirection == "3" or randirection == "5":
            valid = "No"
            return valid
        else:
            valid = "Yes"
            return valid

    def goalState(self):
        #what the robot is working towards
        if all(status in ["1", "3"] for row in self.env.EnvGen for status in row):
            print("Everything has been delivered.")
            print(f"Total moves =  {self.totalmoves} \n Total deliveries = {self.deliveries}")
            print(f"Total Memory = {self.totalmemory}")
        return all(status in ["1", "3"] for row in self.env.EnvGen for status in row)

    def drawTable(self):
        #former viewed table. draws a neater table, easier viewing.
        i = 0
        while i <= self.size:
            #iterates through each location in envgen and changes a number into what it represents
            j = 0
            while j <= self.size:
                if self.env.EnvGen[i][j] == "1":
                    self.Table[i][j] = "Clear   "
                elif self.env.EnvGen[i][j] == "2":
                    self.Table[i][j] = "Delivery"
                    self.totaldeliveries+=1
                elif self.env.EnvGen[i][j] == "3":
                    self.Table[i][j] = "Blockade"
                elif self.env.EnvGen[i][j] == "4":
                    self.Table[i][j] = "Road End"
                elif self.env.EnvGen[i][j] == "5":
                    self.Table[i][j] = "RoadStart"
                j+=1
            i+=1
            self.Table[self.ylocation][self.xlocation] = "Robot   "

    def drawNumberTable(self):
        i = 0
        while i <= self.size:
            j = 0
            while j <= self.size:
                self.NumberTable[i][j] = self.env.EnvGen[i][j]
                j+=1
            i+=1
            self.NumberTable[self.ylocation][self.xlocation] = "X"
                
    def drawDynamicTable(self):
        if self.overallsteps == 1:
            i = 0
            while i <= self.size:
                j = 0
                while j <= self.size:
                    if self.env.EnvGen[i][j] == "2":
                        self.dynamicTable[i][j] = "Delivery"
                    j+=1
                i+=1
        self.dynamicTable[self.ylocation][self.xlocation] = "Robot   "
        if self.xlocation > 0:
            #look left
            self.dynamicTableSight(self.ylocation, self.xlocation - 1)
        if self.xlocation < self.size:
            #look right
            self.dynamicTableSight(self.ylocation, self.xlocation + 1)
        if self.ylocation > 0:
            #look up
            self.dynamicTableSight(self.ylocation - 1, self.xlocation)
        if self.ylocation < self.size:
            #look down
            self.dynamicTableSight(self.ylocation + 1, self.xlocation)
        if self.ylocation > 0 and self.xlocation > 0:
            #look upleft
            self.dynamicTableSight(self.ylocation - 1, self.xlocation - 1)
        if self.ylocation > 0 and self.xlocation < self.size:
            #look upright
            self.dynamicTableSight(self.ylocation - 1, self.xlocation + 1)
        if self.ylocation < self.size and self.xlocation > 0:
            #look downleft
            self.dynamicTableSight(self.ylocation + 1, self.xlocation - 1)
        if self.ylocation < self.size and self.xlocation < self.size:
            #look downright
            self.dynamicTableSight(self.ylocation + 1, self.xlocation + 1)

    def dynamicTableSight(self, i, j):
        block = "0"
        if self.env.EnvGen[i][j] == "1":
            self.dynamicTable[i][j] = "Clear   "
        elif self.env.EnvGen[i][j] == "2":
            self.dynamicTable[i][j] = "Delivery"
        elif self.env.EnvGen[i][j] == "3":
            num = random.choice(["1", "2", "3"])
            if num == "1":
                block = "Building"
            elif num == "2":
                block = "Car     "
            elif num == "3":
                block = "Roadwork"
            self.dynamicTable[i][j] = block
        elif self.env.EnvGen[i][j] == "4":
            self.dynamicTable[i][j] = "RoadEnd"
        elif self.env.EnvGen[i][j] == "5":
            self.dynamicTable[i][j] = "RoadStart"
    
    def run(self):
        self.overallsteps = 0
        self.drawTable()
        print(f"Initial Environment: \n{np.matrix(self.Table)}\n Total Deliveries: {self.totaldeliveries}.\n")
        print("Key: \nX = Robot \n1 = No delivery \n2 = Delivery \n3 = Blockade \n")
        while not self.goalState():
            self.moved = "No"
            self.overallsteps+=1
            print(f"Step {self.overallsteps}:")
            perception = self.percept()
            x = self.xlocation
            y = self.ylocation
            self.act(perception)
            if self.moved == "Yes":
                print(f"Moving to [{self.xlocation},{self.ylocation}].")
            self.drawDynamicTable()
            print(f"Environment State: \n{np.matrix(self.dynamicTable)}\nRobot coordinates: [{self.xlocation},{self.ylocation}].\n Deliveries: {self.deliveries}.\n")
            print(f"Memory: {self.memory}")
            #robot can only do 500 steps just in case, so it doesn't run forever if there's a bug.
            if self.overallsteps == 500:
                break

In [5]:
env = EnvironmentClass()
agent = Robot(env)
agent.run()

How big would you like the environment to be? (2-8)


 8
Do you want deliveries to be randomized or picked?
 1. Randomized.
2. Picked.
 1


Would you like to choose or randomize the starting position? 
1: Choose
2: Randomize.


 2



Robot started in [6,0].
Initial Environment: 
[['Clear   ' 'Delivery' 'Blockade' 'Clear   ' 'Clear   ' 'Clear   '
  'Robot   ' 'Blockade']
 ['Delivery' 'Clear   ' 'Clear   ' 'Clear   ' 'Clear   ' 'Clear   '
  'Blockade' 'Delivery']
 ['Clear   ' 'Delivery' 'Clear   ' 'Delivery' 'Delivery' 'Clear   '
  'Delivery' 'Delivery']
 ['Clear   ' 'Delivery' 'Clear   ' 'Clear   ' 'Clear   ' 'Delivery'
  'Clear   ' 'Delivery']
 ['Clear   ' 'Clear   ' 'Delivery' 'Delivery' 'Delivery' 'Delivery'
  'Blockade' 'Clear   ']
 ['Clear   ' 'Clear   ' 'Clear   ' 'Clear   ' 'Clear   ' 'Clear   '
  'Clear   ' 'Clear   ']
 ['Delivery' 'Clear   ' 'Clear   ' 'Clear   ' 'Clear   ' 'Delivery'
  'Delivery' 'Delivery']
 ['Clear   ' 'Clear   ' 'Clear   ' 'Clear   ' 'Delivery' 'Clear   '
  'Clear   ' 'Delivery']]
 Total Deliveries: 22.

Key: 
X = Robot 
1 = No delivery 
2 = Delivery 
3 = Blockade 

Step 1:
Testing - 7
Delivering at [6,0].
Testing - 7
Environment State: 
[['Unknown ' 'Delivery' 'Unknown ' 'Unknown ' 'U